In [1]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor, T5Model
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

##### Dependencies
!pip install transformers
!pip3 install torch torchvision torchaudio
!pip install sentencepiece

In [2]:
# !pip install transformers
# !pip3 install torch torchvision torchaudio
# !pip install sentencepiece

In [3]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the CPU


In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small',
                                             return_dict=True)
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)
#model.to(dev)

In [3]:
train_df=pd.read_csv('data_cleaned.csv',index_col=0)

train_df=train_df.sample(n=3000)
train_df['title']= train_df['title'].astype('str')
train_df['description']= train_df['description'].astype('str')

In [4]:
train_text, val_text, train_desc,val_desc = train_test_split(train_df['title'],train_df['description'],test_size = 0.3,random_state=1997)

In [5]:
train__text_encodings = tokenizer(train_text.to_list(), truncation=True, padding='max_length',max_length=10)
train__desc_encodings = tokenizer(train_desc.to_list(), truncation=True, padding='max_length',max_length=20)

val__text_encodings = tokenizer(val_text.to_list(), truncation=True, padding='max_length',max_length=10)
val__desc_encodings = tokenizer(val_desc.to_list(), truncation=True, padding='max_length',max_length=20)

In [6]:
from torch.utils.data import TensorDataset, DataLoader

train__text_seq = torch.tensor(train__text_encodings['input_ids'])
train_text_mask = torch.tensor(train__text_encodings['attention_mask'])
train__desc_seq = torch.tensor(train__desc_encodings['input_ids'])
train_desc_mask = torch.tensor(train__desc_encodings['attention_mask'])
train__text_dataset = TensorDataset(train__text_seq,train_text_mask,train__desc_seq,train_desc_mask)





val__text_seq = torch.tensor(val__text_encodings['input_ids'])
val_text_mask = torch.tensor(val__text_encodings['attention_mask'])
val__desc_seq = torch.tensor(val__desc_encodings['input_ids'])
val_desc_mask = torch.tensor(val__desc_encodings['input_ids'])



train__text_loader = DataLoader(train__text_dataset, batch_size=4, shuffle=True)


In [14]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [7]:
#Training POC


model.train()
num_of_epochs=3
loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0


    for batch in train__text_loader:
        text_id,text_mask,desc_id,desc_mask = batch

    # clear out the gradients of all Variables 
        optimizer.zero_grad()

    # Forward propogation
        outputs = model(input_ids=text_id, labels=desc_id)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        # if i%10 ==0:      
        #     loss_per_10_steps.append(loss_num)
        # out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
        loss.backward()

    #updating the params
        optimizer.step()
    

    print('Epoch: {} , = loss: {}'.format(epoch,loss))
torch.save(model.state_dict(), 'stronks_tg.pt')

Running epoch: 1
Epoch: 1 , = loss: 5.218601703643799
Running epoch: 2
Epoch: 2 , = loss: 3.2706024646759033
Running epoch: 3
Epoch: 3 , = loss: 3.660130262374878


In [ ]:
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))